<a href="https://colab.research.google.com/github/bhosalevivek04/LP3/blob/main/LP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Practical l: Write a program non-recursive and recursive program to calculate Fibonacci numbers and analyze their time and space complexity.
def fibonacci(n):
    step_count = 0
    if n == 0:
        return 0, step_count
    elif n == 1:
        return 1, step_count

    a, b = 0, 1
    step_count += 2  # for initial assignments a and b

    for i in range(2, n + 1):
        a, b = b, a + b
        step_count += 1  # for each iteration
        step_count += 2  # for the assignments inside the loop

    return b, step_count

# Example usage:
n = int(input("Enter the Fibonacci term you want to find: "))
fib_number, steps = fibonacci(n)
print(f"Fibonacci number at position {n} is {fib_number}")
print(f"Total steps taken: {steps}")

Enter the Fibonacci term you want to find: 10
Fibonacci number at position 10 is 55
Total steps taken: 29


In [7]:
# Practical 2: Write a program to implement Huffman Encoding using a greedy strategy.
class Job:
    def __init__(self, job_id, deadline, profit):
        self.job_id = job_id
        self.deadline = deadline
        self.profit = profit

def job_sequencing_with_deadlines(jobs):
    # Sort jobs based on decreasing order of profit
    jobs.sort(key=lambda x: x.profit, reverse=True)

    # Find the maximum deadline to create a time slot array
    max_deadline = max(job.deadline for job in jobs)
    time_slots = [-1] * (max_deadline + 1)  # Array to track free time slots

    total_profit = 0
    job_sequence = []

    # Iterate over the sorted jobs
    for job in jobs:
        # Find a free time slot for this job, starting from the latest slot before its deadline
        for t in range(min(job.deadline, max_deadline), 0, -1):
            if time_slots[t] == -1:  # If the time slot is free
                time_slots[t] = job.job_id  # Assign the job to this time slot
                total_profit += job.profit  # Add profit to the total
                job_sequence.append(job.job_id)  # Add job to the sequence
                break  # Move to the next job after assigning

    return job_sequence, total_profit

# Example usage:
jobs = [
    Job('Job1', 2, 100),
    Job('Job2', 1, 19),
    Job('Job3', 2, 27),
    Job('Job4', 1, 25),
    Job('Job5', 3, 15)
]

job_sequence, total_profit = job_sequencing_with_deadlines(jobs)
print(f"Job sequence: {job_sequence}")
print(f"Total profit: {total_profit}")

Job sequence: ['Job1', 'Job3', 'Job5']
Total profit: 142


In [8]:
# Practical 3: Write a program to solve a fractional Knapsack problem using a greedy method.
class Item:
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight

def fractional_knapsack(capacity, items):
    # Sort items by value-to-weight ratio in decreasing order
    items.sort(key=lambda x: x.value / x.weight, reverse=True)

    total_value = 0  # Total value in the knapsack

    for item in items:
        if capacity - item.weight >= 0:
            # If the whole item can be added
            capacity -= item.weight
            total_value += item.value
        else:
            # If only part of the item can be added
            total_value += item.value * (capacity / item.weight)
            break  # Knapsack is full

    return total_value

# Example usage
if __name__ == "__main__":
    items = [Item(60, 10), Item(100, 20), Item(120, 30)]  # Example items
    capacity = 50  # Knapsack capacity
    max_value = fractional_knapsack(capacity, items)
    print(f"Maximum value in the knapsack: {max_value:.2f}")

Maximum value in the knapsack: 240.00


In [9]:
# Practical 4: Write a program to solve a 0-1 Knapsack problem using dynamic programming or branch and bound strategy.

# Code: 0-1 Knapsack using Dynamic Programming
def knapsack_dp(weights, values, capacity):
    n = len(weights)
    # Create a 2D array to store the maximum value for each capacity and item count
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]

    # Build the dp array
    for i in range(1, n + 1):
        for w in range(1, capacity + 1):
            if weights[i - 1] <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - weights[i - 1]] + values[i - 1])
            else:
                dp[i][w] = dp[i - 1][w]

    # The value in the last cell will be the maximum value for the knapsack
    return dp[n][capacity]

# Example usage
values = [60, 100, 120]   # Value of each item
weights = [10, 20, 30]    # Weight of each item
capacity = 50             # Capacity of knapsack

max_value = knapsack_dp(weights, values, capacity)
print(f"Maximum value in the knapsack: {max_value}")

Maximum value in the knapsack: 220


In [10]:
# Code: 0-1 Knapsack using Branch and Bound
class Item:
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight
        self.ratio = value / weight

class Node:
    def __init__(self, level, profit, weight, bound):
        self.level = level
        self.profit = profit
        self.weight = weight
        self.bound = bound

def bound(node, n, capacity, items):
    if node.weight >= capacity:
        return 0
    profit_bound = node.profit
    j = node.level + 1
    total_weight = node.weight

    while j < n and total_weight + items[j].weight <= capacity:
        total_weight += items[j].weight
        profit_bound += items[j].value
        j += 1

    if j < n:
        profit_bound += (capacity - total_weight) * items[j].ratio

    return profit_bound

def knapsack_branch_and_bound(capacity, items, n):
    items.sort(key=lambda x: x.ratio, reverse=True)
    queue = []
    u = Node(-1, 0, 0, 0)
    u.bound = bound(u, n, capacity, items)
    max_profit = 0
    queue.append(u)

    while queue:
        u = queue.pop(0)
        if u.level == -1:
            v = Node(0, 0, 0, 0)
        if u.level == n - 1:
            continue

        v = Node(u.level + 1, u.profit + items[v.level].value, u.weight + items[v.level].weight, 0)

        if v.weight <= capacity and v.profit > max_profit:
            max_profit = v.profit

        v.bound = bound(v, n, capacity, items)
        if v.bound > max_profit:
            queue.append(v)

        v = Node(u.level + 1, u.profit, u.weight, 0)
        v.bound = bound(v, n, capacity, items)
        if v.bound > max_profit:
            queue.append(v)

    return max_profit

# Example usage
values = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50
items = [Item(values[i], weights[i]) for i in range(len(values))]
n = len(items)

max_profit = knapsack_branch_and_bound(capacity, items, n)
print(f"Maximum profit in the knapsack: {max_profit}")

Maximum profit in the knapsack: 220


In [11]:
# 5. Mini Project
import time
import random
from threading import Thread

# Merge Sort Implementation
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        L = arr[:mid]
        R = arr[mid:]

        merge_sort(L)
        merge_sort(R)

        i = j = k = 0
        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1

# Multithreaded Merge Sort Implementation
def threaded_merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        L = arr[:mid]
        R = arr[mid:]

        left_thread = Thread(target=merge_sort, args=(L,))
        right_thread = Thread(target=merge_sort, args=(R,))

        # Start the threads
        left_thread.start()
        right_thread.start()

        # Wait for both threads to finish
        left_thread.join()
        right_thread.join()

        i = j = k = 0
        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1

# Function to measure time taken by sorting algorithms
def measure_time(sort_function, arr):
    start_time = time.time()
    sort_function(arr)
    end_time = time.time()
    return end_time - start_time

# Main function to compare both algorithms
def compare_merge_sort_algorithms():
    arr_size = 10000  # Adjust the size for testing
    best_case = list(range(arr_size))  # Sorted array (best case for merge sort)
    worst_case = list(range(arr_size, 0, -1))  # Reversed array (worst case for merge sort)

    # Measure time for single-threaded merge sort
    print("Testing Single-threaded Merge Sort:")
    best_time_single = measure_time(merge_sort, best_case.copy())
    worst_time_single = measure_time(merge_sort, worst_case.copy())
    print(f"Best case time (Single-threaded): {best_time_single:.6f} seconds")
    print(f"Worst case time (Single-threaded): {worst_time_single:.6f} seconds")

    # Measure time for multithreaded merge sort
    print("\nTesting Multithreaded Merge Sort:")
    best_time_multi = measure_time(threaded_merge_sort, best_case.copy())
    worst_time_multi = measure_time(threaded_merge_sort, worst_case.copy())
    print(f"Best case time (Multithreaded): {best_time_multi:.6f} seconds")
    print(f"Worst case time (Multithreaded): {worst_time_multi:.6f} seconds")

# Run the comparison
if __name__ == "__main__":
    compare_merge_sort_algorithms()

Testing Single-threaded Merge Sort:
Best case time (Single-threaded): 0.037866 seconds
Worst case time (Single-threaded): 0.035687 seconds

Testing Multithreaded Merge Sort:
Best case time (Multithreaded): 0.053634 seconds
Worst case time (Multithreaded): 0.065706 seconds
